<a href="https://colab.research.google.com/github/Keerthi001/EVA-Assignment/blob/main/Copy_of_Copy_of_s4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [70]:
# Model paramters: 11730, accuray: >99.4

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(1, 4, 3, padding=1),
            nn.ReLU(),
            # nn.Conv2d(4, 16, 3, padding=1),
            # nn.ReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.50),
            nn.Conv2d(4, 16, 3, padding=1),
            nn.ReLU(),
            # nn.Conv2d(16, 32, 3, padding=1),
            # nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.50),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.Conv2d(32, 10, 3),
            nn.AvgPool2d(3, 2),
        )
 
    def forward(self, x):
        x = self.conv(x)
        
        x = x.view(-1, 10)
        x = F.log_softmax(x, dim=1)
        return x


In [71]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
              ReLU-2            [-1, 4, 28, 28]               0
       BatchNorm2d-3            [-1, 4, 28, 28]               8
         MaxPool2d-4            [-1, 4, 14, 14]               0
           Dropout-5            [-1, 4, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]             592
              ReLU-7           [-1, 16, 14, 14]               0
       BatchNorm2d-8           [-1, 16, 14, 14]              32
         MaxPool2d-9             [-1, 16, 7, 7]               0
          Dropout-10             [-1, 16, 7, 7]               0
           Conv2d-11             [-1, 32, 5, 5]           4,640
             ReLU-12             [-1, 32, 5, 5]               0
           Conv2d-13             [-1, 10, 3, 3]           2,890
        AvgPool2d-14             [-1, 1

In [72]:
torch.manual_seed(1)
batch_size = 128
 
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        #transforms.RandomRotation(30),
                        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ColorJitter(brightness=0.2, contrast=0.2),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [73]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
 
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
 
 
 
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
 
    test_loss /= len(test_loader.dataset)
 
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

In [74]:
import torch.optim as optim
 
model = Net().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer= optim.Adam(model.parameters(), lr= 0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2) # Learning rate scheduler with patience = 2
 
for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)

    
    print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))
    scheduler.step(test_loss)

loss=0.584715723991394 batch_id=0:   0%|          | 1/469 [00:00<01:08,  6.84it/s]


Test set: Average loss: 0.1671, Accuracy: 9487/10000 (94.87%)

Epoch-1 lr: 0.01


loss=0.3929958939552307 batch_id=0:   0%|          | 1/469 [00:00<01:10,  6.66it/s]


Test set: Average loss: 0.1151, Accuracy: 9641/10000 (96.41%)

Epoch-2 lr: 0.01


loss=0.35271069407463074 batch_id=0:   0%|          | 1/469 [00:00<01:09,  6.75it/s]

...


Test set: Average loss: 0.0714, Accuracy: 9761/10000 (97.61%)

Epoch-12 lr: 0.001


loss=0.17257538437843323 batch_id=0:   0%|          | 1/469 [00:00<01:17,  6.04it/s]


Test set: Average loss: 0.0694, Accuracy: 9769/10000 (97.69%)

Epoch-13 lr: 0.001


loss=0.3187442421913147 batch_id=468: 100%|██████████| 469/469 [01:11<00:00,  6.52it/s]



Test set: Average loss: 0.0710, Accuracy: 9768/10000 (97.68%)

Epoch-14 lr: 0.001
